In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('../data/processed/merged_dataset_cleaned.csv', sep = ";", index_col = 'utc_timestamp', parse_dates = True)

In [13]:
df = df.rename(columns={
    "DE_load_actual_entsoe_transparency": "actual_load",
    "DE_solar_capacity": "solar_capacity",
    "DE_solar_generation_actual": "solar_generation",
    "DE_wind_capacity": "wind_capacity",
    "DE_wind_generation_actual": "wind_generation",
    "DE_wind_offshore_capacity": "wind_offshore_capacity",
    "DE_wind_offshore_generation_actual": "wind_offshore_generation",
    "DE_wind_onshore_capacity": "wind_onshore_capacity",
    "DE_wind_onshore_generation_actual": "wind_onshore_generation",
    "DE_temperature": "temperature"
})

df = df[[
    "actual_load", "solar_capacity", "solar_generation", "wind_capacity", "wind_generation",
    "wind_offshore_capacity", "wind_offshore_generation", "wind_onshore_capacity", "wind_onshore_generation",
    "temperature", "hour", "is_daylight", "month"
]]

df["year"] = df.index.year
df["weekday"] = df.index.weekday
df["weekend_flag"] = df["weekday"].apply(lambda x: 1 if x >= 5 else 0)

In [27]:
import holidays

# Fix: Convert timezone-aware index to naive before matching with holidays
df["holiday_flag"] = df.index.tz_convert(None).normalize().isin(de_holidays).astype(int)

In [30]:
# check edge case: 2019-10-03 (Tag der Deutschen Einheit)
print(df.loc["2019-10-03", "holiday_flag"])
print(df.loc["2019-10-02", "holiday_flag"])

utc_timestamp
2019-10-03 00:00:00+00:00    1
2019-10-03 00:15:00+00:00    1
2019-10-03 00:30:00+00:00    1
2019-10-03 00:45:00+00:00    1
2019-10-03 01:00:00+00:00    1
                            ..
2019-10-03 22:45:00+00:00    1
2019-10-03 23:00:00+00:00    1
2019-10-03 23:15:00+00:00    1
2019-10-03 23:30:00+00:00    1
2019-10-03 23:45:00+00:00    1
Name: holiday_flag, Length: 96, dtype: int64
utc_timestamp
2019-10-02 00:00:00+00:00    0
2019-10-02 00:15:00+00:00    0
2019-10-02 00:30:00+00:00    0
2019-10-02 00:45:00+00:00    0
2019-10-02 01:00:00+00:00    0
                            ..
2019-10-02 22:45:00+00:00    0
2019-10-02 23:00:00+00:00    0
2019-10-02 23:15:00+00:00    0
2019-10-02 23:30:00+00:00    0
2019-10-02 23:45:00+00:00    0
Name: holiday_flag, Length: 96, dtype: int64


In [31]:
df.to_csv('../data/processed/merged_dataset_featurized.csv', sep = ";")